In [11]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import string
import transformers
from pandas import DataFrame, Series
from typing import Tuple
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
from tensorflow.python.keras.layers import Input, Dropout, Dense
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.metrics import BinaryAccuracy, Precision, Recall

MARKER_COLUMN_NAME = 'MARKER'
CLASSE_COLUMN_NAME = 'DES_CLASSE'
EMENTA_COLUMN_NAME = 'TXT_EMENTA'
METRICS = [
    BinaryAccuracy(name='accuracy'),
    Precision(name='precision'),
    Recall(name='recall')
]


def separate_datasets(df: DataFrame, classe: string) -> Tuple[DataFrame, DataFrame]:
    print("Separando datasets")
    df_classe = df[df[CLASSE_COLUMN_NAME] == classe]
    print("Dataset da classe-alvo:", df_classe.shape)
    df_outrasClasses = df[df[CLASSE_COLUMN_NAME] != classe]
    print("Dataset das outras classes:", df_outrasClasses.shape)
    return df_classe, df_outrasClasses


def create_downsample_df(df_classe: DataFrame, df_outrasClasses: DataFrame) -> DataFrame:
    quantidadeDeRegistros = df_classe.shape[0]
    print("Criando um dataset das outras classes com ",
          quantidadeDeRegistros, " registros")
    return df_outrasClasses.sample(quantidadeDeRegistros)


def create_balanced_df(df_classe: DataFrame, df_outrasClasses: DataFrame) -> DataFrame:
    df_outrasClasses_downsampled = create_downsample_df(
        df_classe, df_outrasClasses)
    print("Juntando os datasets...")
    df_balanced = pd.concat([df_outrasClasses_downsampled, df_classe])
    print('Verificando a quantidade de classes existentes no DataFrame após o balanceamento:')
    print(df_balanced[CLASSE_COLUMN_NAME].value_counts())
    return df_balanced


def mark_classe(df_balanced: DataFrame, classe: string):
    print("Incluindo marcador na classe ", classe)
    df_balanced[MARKER_COLUMN_NAME] = df_balanced[CLASSE_COLUMN_NAME].apply(
        lambda x: 1 if x == classe else 0)
    print("Verificando marcadores:")
    print(df_balanced.sample(5))


def create_keras_model() -> Model:
    bert_preprocess = hub.KerasLayer(AutoTokenizer.from_pretrained(
        'neuralmind/bert-large-portuguese-cased'))
    bert_encoder = hub.KerasLayer(AutoModel.from_pretrained(
        'neuralmind/bert-large-portuguese-cased'))

    # Bert layers
    text_input = Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)

    # Neural network layers
    l = Dropout(0.1, name="dropout")(outputs['pooled_output'])
    l = Dense(1, activation='sigmoid', name="output")(l)

    # Use inputs and outputs to construct a final model
    model = Model(inputs=[text_input], outputs=[l])
    print(model.summary)
    return model


def evaluate_model(model: Model, X_test: list):
    y_predicted = model.predict(X_test)
    y_predicted = y_predicted.flatten()
    y_predicted = np.where(y_predicted > 0.5, 1, 0)
    print(y_predicted)


def train_class(df: DataFrame, classe: string,  balanceRatio=0) -> Model:
    print('Verificando a quantidade de classes existentes no DataFrame')
    print(df[CLASSE_COLUMN_NAME].value_counts())
    df_classe, df_outrasClasses = separate_datasets(
        df, classe)
    df_balanced = create_balanced_df(df_classe, df_outrasClasses)
    mark_classe(df_balanced, classe)
    ementas = df_balanced[EMENTA_COLUMN_NAME]
    markers = df_balanced[MARKER_COLUMN_NAME]
    X_train, X_test, y_train, y_test = train_test_split(
        ementas, markers, stratify=markers)
    model = create_keras_model()
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=METRICS)
    model.fit(X_train, y_train, epochs=10)
    evaluate_model(model, X_test)
    nomeArquivoModelo = classe + '_bert_model.h5'
    model.save(nomeArquivoModelo)
    return model

In [3]:
def get_leis_classificadas() -> DataFrame :
    classes_original = pd.read_excel("../dados/SistemaDeClassificacao.xlsx", "Select viw_classificacao_arvore")
    classes_original.head()
    classes_raiz = classes_original.query("NUM_NIVEL == 2").filter(["COD_CLASSE", "DES_NOME_PREFERIDO"])
    classes_raiz.rename(columns={"DES_NOME_PREFERIDO": "DES_CLASSE"}, inplace=True)
    classes_raiz
    leis_classificadas_original = pd.read_excel("../dados/ClassificacaoDeLeisOrdinarias-LeisComplementares-e-DecretosNumerados-Desde1900.xlsx", "Select mvw_u03_prc_doc_tema")
    leis_classificadas_original["DES_CLASSE_RAIZ"] = leis_classificadas_original["DES_CLASSE_HIERARQUIA"].apply(lambda hierarquia : hierarquia.split(" / ")[1])
    leis_classificadas_original.head()
    leis_original = pd.read_excel("../dados/LeisOrdinarias-LeisComplementare-e-DecretosNumeradosComClassificacaoDesde1900.xlsx", "Select mvw_s01_documento")
    leis_original.rename(columns={"DBMS_LOB.SUBSTR(S01.TXT_EMENTA": "TXT_EMENTA"}, inplace=True)
    leis_original.drop(columns="   ", inplace=True)
    leis_original.head()
    leis_original.shape[0]
    leis = leis_original.merge(leis_classificadas_original.filter(["COD_PROCESSO_DOCUMENTO","DES_CLASSE_RAIZ"]), left_on="COD_DOCUMENTO", right_on="COD_PROCESSO_DOCUMENTO", how="left")
    leis = leis.merge(classes_raiz, left_on="DES_CLASSE_RAIZ", right_on="DES_CLASSE", how="left")
    leis.drop(columns=["COD_PROCESSO_DOCUMENTO", "DES_CLASSE_RAIZ"], inplace=True)
    leis.head()
    leis.shape[0]
    leis.drop_duplicates(inplace=True)
    temp = leis[["COD_DOCUMENTO", "COD_CLASSE", "DES_CLASSE"]].groupby("COD_DOCUMENTO")
    temp.filter(lambda x: len(x) > 1)
    leis.shape[0]
    leis_classificadas = leis[leis["COD_CLASSE"] >= 0]
    return leis_classificadas

Código que efetuará o treinamento

In [12]:
df_leis_classificadas = get_leis_classificadas()
model = train_class(df_leis_classificadas, 'Meio Ambiente')

Verificando a quantidade de classes existentes no DataFrame
Orçamento Público                             11192
Infraestrutura                                 2508
Política Social                                 877
Administração Pública                           783
Honorífico                                      671
Economia e Desenvolvimento                      644
Jurídico                                        381
Soberania, Defesa Nacional e Ordem Pública      166
Organização do Estado                           130
Meio Ambiente                                    86
Name: DES_CLASSE, dtype: int64
Separando datasets
Dataset da classe-alvo: (86, 6)
Dataset das outras classes: (17352, 6)
Criando um dataset das outras classes com  86  registros
Juntando os datasets...
Verificando a quantidade de classes existentes no DataFrame após o balanceamento:
Meio Ambiente                                 86
Orçamento Público                             47
Infraestrutura                        

Some weights of the model checkpoint at neuralmind/bert-large-portuguese-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ValueError: Exception encountered when calling layer "keras_layer" "                 f"(type KerasLayer).

text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

Call arguments received by layer "keras_layer" "                 f"(type KerasLayer):
  • inputs=<KerasTensor: shape=(None,) dtype=string (created by layer 'text')>
  • training=None